# 연습문제1
- 기상 관측 이래, 서울의 최고 기온이 가장 높았던 날은 언제였고, 몇도인가요?

- 역사적으로 일교차가 가장 큰 날짜는 몇년 몇월 몇일 인가요?

- 1년 중 평균적으로 일교차가 가장 큰 날짜는 몇월 몇일 인가요?

- 가장 덥다고 알려진 대구보다 서울이 더 더운날이 가장 많은 연도는 언제인가요?

## 데이터 불러오기

In [2]:
import numpy as np
np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.1f}".format(x)})

In [447]:
seoul = np.loadtxt('data/서울.csv', delimiter=',', skiprows=8,
                   dtype=np.object)
daegu = np.loadtxt('data/대구.csv', delimiter=',', skiprows=8,
                  dtype=np.object)

In [448]:
seoul[:,0] = list(map(lambda x: x[:4]+x[5:7]+x[8:10], seoul[:,0]))
daegu[:,0] = list(map(lambda x: x[:4]+x[5:7]+x[8:10], daegu[:,0]))

In [449]:
seoul

array([['19071001', '108', '13.5', '7.9', '20.7'],
       ['19071002', '108', '16.2', '7.9', '22'],
       ['19071003', '108', '16.2', '13.1', '21.3'],
       ...,
       ['20200905', '108', '23.3', '19.2', '27.3'],
       ['20200906', '108', '21.7', '19.5', '24.4'],
       ['20200907', '108', '19.2', '17.5', '20.6']], dtype=object)

In [450]:
# 결측치 있는 날짜 제거
del_idx = []
for i in range(len(seoul)):
    if '' in seoul[i]:
        del_idx.append(i)
seoul = np.delete(seoul, del_idx, axis=0)

del_idx = []
for i in range(len(daegu)):
    if '' in daegu[i]:
        del_idx.append(i)
daegu = np.delete(daegu, del_idx, axis=0)

In [451]:
seoul = seoul.astype(np.float64)
daegu = daegu.astype(np.float64)

##  기상 관측 이래, 서울의 최고 기온이 가장 높았던 날은 언제였고, 몇도인가요? 

### 정답 

In [452]:
high_temp = seoul[seoul[:,-1].argmax()]
print(f'날짜: {str(high_temp[0])[:-2]}')
print(f'온도: {high_temp[-1]}')

날짜: 20180801
온도: 39.6


##  역사적으로 일교차가 가장 큰 날짜는 몇년 몇월 몇일 인가요?

### 풀이 

In [453]:
temp_range = seoul[:,-1] - seoul[:,-2]

In [454]:
seoul = np.concatenate((seoul, temp_range.reshape(-1,1)), axis=1)

### 정답 

In [455]:
high_range = seoul[temp_range.argmax()]
print(f'날짜: {str(high_range[0])[:-2]}')
print(f'일교차: {temp_range.max()}')

날짜: 19420419
일교차: 21.8


## 1년 중 평균적으로 일교차가 가장 큰 날짜는 몇월 몇일 인가요?

### 풀이

In [456]:
month_day = np.array(list(map(lambda x: str(x)[4:8], seoul[:,0])))

In [457]:
seoul = np.concatenate((seoul, month_day.reshape(-1,1)), axis=1)

In [458]:
seoul = seoul.astype(np.float64)

In [459]:
mean_temp = []
for md in np.unique(seoul[:,6]):
    mean_temp.append((md,seoul[seoul[:,6] == md][:,5].mean()))
mean_temp = np.array(mean_temp)   

### 정답

In [460]:
high_md_temp = mean_temp[mean_temp[:,1].argmax()]
print(f'월,일: {str(high_md_temp[0])}')
print(f'평균일교차: {high_md_temp[1]}')

월,일: 429.0
평균일교차: 11.953636363636363


##  가장 덥다고 알려진 대구보다 서울이 더 더운날이 가장 많은 연도는 언제인가요? 

In [461]:
seoul = np.delete(seoul, [5,6], axis=1)

In [464]:
d_temp = []
for date in seoul[:,0]:
    if list(daegu[daegu[:,0] == date]):
        d_temp.append(daegu[daegu[:,0]==date][0,-1])
    else:
        d_temp.append('')

In [466]:
seoul = np.concatenate((seoul, np.array(d_temp).reshape(-1,1)), axis=1)

In [467]:
del_idx = []
for i in range(len(seoul)):
    if '' in seoul[i]:
        del_idx.append(i)
seoul = np.delete(seoul, del_idx, axis=0)

In [477]:
high_temp_seoul = seoul[seoul[:,-2] > seoul[:,-1]]

In [479]:
year = np.array(list(map(lambda x: str(x)[:4], high_temp_seoul[:,0])))

In [480]:
high_temp_seoul = np.concatenate((high_temp_seoul, year.reshape(-1,1)), axis=1)

In [521]:
mean_temp = []
for y in np.unique(high_temp_seoul[:,-1]):
    mean_temp.append((y,len(high_temp_seoul[high_temp_seoul[:,-1] == y])))
mean_temp = np.array(mean_temp, dtype=np.int)

### 정답 

In [527]:
result = mean_temp[mean_temp[:,1] ==  mean_temp[:,1].max()]
for i in range(len(result)):
    print(f'연도: {str(result[i,0])} 년')
    print(f'더운날: {int(result[i,1])} 일')

연도: 1930 년
더운날: 159 일
연도: 2016 년
더운날: 159 일


# 연습문제2
- 사용자가 평가한 모든 영화의 전체 평균 평점
- 각 사용자별 평균 평점
- 각 영화별 평균 평점
- 평균 평점이 가장 높은 영화의 제목

## 데이터 불러오기 

In [560]:
import csv

In [561]:
movie_list = []
f = open('data/movies.csv', 'r', encoding='utf-8')
lines = csv.reader(f)
for line in lines:
    movie_list.append(line)
f.close()

In [564]:
movie_list.pop(0)

['movieId', 'title', 'genres']

In [590]:
movie_arr = np.array(movie_list)

In [567]:
rating = np.loadtxt('data/ratings.csv', delimiter=',', skiprows=1,
                   dtype=np.object, encoding='cp949')

## 사용자가 평가한 모든 영화의 전체 평균 평점

### 정답 

In [573]:
rating[:,2].astype(np.float64).mean()

3.501556983616962

## 각 사용자별 평균 평점

In [574]:
user_dict = {user: 0 for user in np.unique(rating[:,0])}

In [576]:
for user in user_dict.keys():
    user_dict[user] = rating[rating[:,0] == user][:,2].astype(np.float64).mean()

### 정답 

In [579]:
for user in user_dict.items():
    print(f'ID: {user[0]} -> 평균 평점: {round(user[1],3)} 점')

ID: 1 -> 평균 평점: 4.366 점
ID: 10 -> 평균 평점: 3.279 점
ID: 100 -> 평균 평점: 3.946 점
ID: 101 -> 평균 평점: 3.557 점
ID: 102 -> 평균 평점: 3.357 점
ID: 103 -> 평균 평점: 3.907 점
ID: 104 -> 평균 평점: 3.507 점
ID: 105 -> 평균 평점: 4.116 점
ID: 106 -> 평균 평점: 4.439 점
ID: 107 -> 평균 평점: 3.912 점
ID: 108 -> 평균 평점: 3.987 점
ID: 109 -> 평균 평점: 3.22 점
ID: 11 -> 평균 평점: 3.781 점
ID: 110 -> 평균 평점: 3.725 점
ID: 111 -> 평균 평점: 3.34 점
ID: 112 -> 평균 평점: 3.592 점
ID: 113 -> 평균 평점: 3.647 점
ID: 114 -> 평균 평점: 3.435 점
ID: 115 -> 평균 평점: 3.768 점
ID: 116 -> 평균 평점: 3.437 점
ID: 117 -> 평균 평점: 3.339 점
ID: 118 -> 평균 평점: 3.818 점
ID: 119 -> 평균 평점: 4.177 점
ID: 12 -> 평균 평점: 4.391 점
ID: 120 -> 평균 평점: 3.409 점
ID: 121 -> 평균 평점: 3.293 점
ID: 122 -> 평균 평점: 4.546 점
ID: 123 -> 평균 평점: 4.036 점
ID: 124 -> 평균 평점: 3.99 점
ID: 125 -> 평균 평점: 3.86 점
ID: 126 -> 평균 평점: 3.289 점
ID: 127 -> 평균 평점: 3.114 점
ID: 128 -> 평균 평점: 4.364 점
ID: 129 -> 평균 평점: 3.921 점
ID: 13 -> 평균 평점: 3.645 점
ID: 130 -> 평균 평점: 3.536 점
ID: 131 -> 평균 평점: 3.442 점
ID: 132 -> 평균 평점: 3.039 점
ID: 133 -> 평균 평점: 3.0 

## 각 영화별 평균 평점

In [625]:
movie_dict = {movie: [] for movie in np.unique(rating[:,1])}

In [626]:
for movie in movie_dict.keys():
    movie_dict[movie].append(rating[rating[:,1] == movie][:,2].astype(np.float64).mean())
    movie_dict[movie].append(movie_arr[movie_arr[:,0] == movie][0,1])

### 정답 

In [622]:
for movie in movie_dict.items():
    print(f'movie: {movie_arr[movie_arr[:,0] == movie[0]][0,1]} -> 평균 평점: {round(movie[1][0],3)} 점')

movie: Toy Story (1995) -> 평균 평점: 3.921 점
movie: GoldenEye (1995) -> 평균 평점: 3.496 점
movie: City Hall (1996) -> 평균 평점: 2.786 점
movie: Human Planet (2011) -> 평균 평점: 4.0 점
movie: Comme un chef (2012) -> 평균 평점: 3.5 점
movie: Movie 43 (2013) -> 평균 평점: 3.5 점
movie: Pervert's Guide to Ideology, The (2012) -> 평균 평점: 3.5 점
movie: Sightseers (2012) -> 평균 평점: 4.5 점
movie: Hansel & Gretel: Witch Hunters (2013) -> 평균 평점: 2.9 점
movie: Jim Jefferies: Fully Functional (EPIX) (2012) -> 평균 평점: 4.5 점
movie: Why Stop Now (2012) -> 평균 평점: 1.5 점
movie: Tabu (2012) -> 평균 평점: 4.0 점
movie: Extreme Measures (1996) -> 평균 평점: 2.5 점
movie: Upside Down (2012) -> 평균 평점: 3.0 점
movie: Liability, The (2012) -> 평균 평점: 3.0 점
movie: Angst  (1983) -> 평균 평점: 3.5 점
movie: Stand Up Guys (2012) -> 평균 평점: 2.5 점
movie: Side Effects (2013) -> 평균 평점: 3.917 점
movie: Identity Thief (2013) -> 평균 평점: 2.875 점
movie: ABCs of Death, The (2012) -> 평균 평점: 3.5 점
movie: Glimmer Man, The (1996) -> 평균 평점: 3.0 점
movie: Beautiful Creatures (2013)

movie: Nebraska (2013) -> 평균 평점: 4.125 점
movie: Enough Said (2013) -> 평균 평점: 3.0 점
movie: Don Jon (2013) -> 평균 평점: 3.125 점
movie: Mood Indigo (L'écume des jours) (2013) -> 평균 평점: 3.5 점
movie: Century of the Self, The (2002) -> 평균 평점: 3.75 점
movie: Crystal Fairy & the Magical Cactus and 2012 (2013) -> 평균 평점: 3.5 점
movie: Normal Life (1996) -> 평균 평점: 3.0 점
movie: Bad Milo (Bad Milo!) (2013) -> 평균 평점: 1.0 점
movie: Runner Runner (2013) -> 평균 평점: 2.25 점
movie: Blue Is the Warmest Color (La vie d'Adèle) (2013) -> 평균 평점: 3.25 점
movie: Get on the Bus (1996) -> 평균 평점: 2.0 점
movie: Cloudy with a Chance of Meatballs 2 (2013) -> 평균 평점: 2.25 점
movie: Shadow Conspiracy (1997) -> 평균 평점: 1.5 점
movie: Captain Phillips (2013) -> 평균 평점: 4.048 점
movie: All Dogs Christmas Carol, An (1998) -> 평균 평점: 4.5 점
movie: Machete Kills (Machete 2) (2013) -> 평균 평점: 4.0 점
movie: Filth (2013) -> 평균 평점: 4.5 점
movie: Jude (1996) -> 평균 평점: 3.071 점
movie: Escape Plan (2013) -> 평균 평점: 4.0 점
movie: Everyone Says I Love You (1

movie: King of Comedy (Hei kek ji wong) (1999) -> 평균 평점: 4.0 점
movie: 13 Sins (2014) -> 평균 평점: 3.5 점
movie: Muppets Most Wanted (2014) -> 평균 평점: 3.333 점
movie: Rebel Without a Cause (1955) -> 평균 평점: 4.1 점
movie: Me and you (io e te) (2012) -> 평균 평점: 4.5 점
movie: Free to Play (2014) -> 평균 평점: 4.5 점
movie: Unknown Known, The (2013) -> 평균 평점: 4.0 점
movie: Streetcar Named Desire, A (1951) -> 평균 평점: 4.475 점
movie: Children of the Corn IV: The Gathering (1996) -> 평균 평점: 4.667 점
movie: The Raid 2: Berandal (2014) -> 평균 평점: 3.929 점
movie: Brain Smasher... A Love Story (Bouncer and the Lady, The) (1993) -> 평균 평점: 2.5 점
movie: The Amazing Spider-Man 2 (2014) -> 평균 평점: 2.636 점
movie: Calvary (2014) -> 평균 평점: 4.0 점
movie: Oculus (2013) -> 평균 평점: 4.5 점
movie: God's Not Dead (2014) -> 평균 평점: 0.5 점
movie: Cold in July (2014) -> 평균 평점: 2.5 점
movie: Rio 2 (2014) -> 평균 평점: 2.0 점
movie: Honest Liar, An (2014) -> 평균 평점: 3.75 점
movie: Loser (1991) -> 평균 평점: 2.0 점
movie: Fading Gigolo (2013) -> 평균 평점: 4.0 점

movie: Copenhagen (2014) -> 평균 평점: 4.5 점
movie: Simpsons: The Longest Daycare, The (2012) -> 평균 평점: 3.667 점
movie: Generation War (2013) -> 평균 평점: 4.0 점
movie: Anne Frank Remembered (1995) -> 평균 평점: 3.9 점
movie: Christmas in Connecticut (1992) -> 평균 평점: 1.5 점
movie: Tin Drum, The (Blechtrommel, Die) (1979) -> 평균 평점: 3.667 점
movie: Leviathan (2014) -> 평균 평점: 2.0 점
movie: Reign of Assassins (2010) -> 평균 평점: 2.5 점
movie: Ruling Class, The (1972) -> 평균 평점: 3.0 점
movie: Zulu (2013) -> 평균 평점: 1.5 점
movie: Mina Tannenbaum (1994) -> 평균 평점: 1.0 점
movie: Tangerines (2013) -> 평균 평점: 4.5 점
movie: Life Partners (2014) -> 평균 평점: 3.0 점
movie: Drive Hard (2014) -> 평균 평점: 1.5 점
movie: New Kids Nitro (2011) -> 평균 평점: 3.0 점
movie: Stalingrad (2013) -> 평균 평점: 3.0 점
movie: Dead Snow 2: Red vs. Dead (2014)  -> 평균 평점: 4.0 점
movie: Dear God (1996) -> 평균 평점: 3.5 점
movie: Volga - Volga (1938) -> 평균 평점: 3.5 점
movie: You Are the Apple of My Eye (2011) -> 평균 평점: 4.0 점
movie: DeadHeads (2011) -> 평균 평점: 3.5 점
movie:

movie: Doctor Strange (2016) -> 평균 평점: 3.705 점
movie: X-Men: Apocalypse (2016) -> 평균 평점: 3.071 점
movie: Untitled Spider-Man Reboot (2017) -> 평균 평점: 4.156 점
movie: Elsa & Fred (2014) -> 평균 평점: 2.0 점
movie: Chungking Express (Chung Hing sam lam) (1994) -> 평균 평점: 4.375 점
movie: Annie Hall (1977) -> 평균 평점: 3.871 점
movie: Right Stuff, The (1983) -> 평균 평점: 3.864 점
movie: Stalker (1979) -> 평균 평점: 3.625 점
movie: Jim Jefferies: I Swear to God (2009) -> 평균 평점: 4.5 점
movie: Boot, Das (Boat, The) (1981) -> 평균 평점: 4.212 점
movie: Tornado! (1996) -> 평균 평점: 2.0 점
movie: Sting, The (1973) -> 평균 평점: 3.977 점
movie: Harold and Maude (1971) -> 평균 평점: 4.288 점
movie: The Blue Lagoon (1949) -> 평균 평점: 3.0 점
movie: In the Name of the King III (2014) -> 평균 평점: 1.0 점
movie: Trust (1990) -> 평균 평점: 4.333 점
movie: Seventh Seal, The (Sjunde inseglet, Det) (1957) -> 평균 평점: 4.222 점
movie: Local Hero (1983) -> 평균 평점: 4.056 점
movie: Cake (2014) -> 평균 평점: 2.75 점
movie: Terminator, The (1984) -> 평균 평점: 3.897 점
movie: Dead 

...

movie: Black Narcissus (1947) -> 평균 평점: 4.0 점
movie: Escape from Alcatraz (1979) -> 평균 평점: 3.786 점
movie: Oliver Twist (1948) -> 평균 평점: 3.833 점
movie: Season of the Witch (2011) -> 평균 평점: 3.5 점
movie: Foxfire (1996) -> 평균 평점: 3.667 점
movie: Chain Reaction (1996) -> 평균 평점: 2.882 점
movie: Shrek 2 (2004) -> 평균 평점: 3.576 점
movie: Amer (2009) -> 평균 평점: 0.5 점
movie: Day After Tomorrow, The (2004) -> 평균 평점: 3.05 점
movie: Cowboys & Aliens (2011) -> 평균 평점: 3.0 점
movie: Raising Helen (2004) -> 평균 평점: 2.75 점
movie: Soul Plane (2004) -> 평균 평점: 3.75 점
movie: Baadasssss! (How to Get the Man's Foot Outta Your Ass) (2003) -> 평균 평점: 3.375 점
movie: Saved! (2004) -> 평균 평점: 3.893 점
movie: Harry Potter and the Prisoner of Azkaban (2004) -> 평균 평점: 3.914 점
movie: Mindhunters (2004) -> 평균 평점: 2.667 점
movie: Matilda (1996) -> 평균 평점: 3.273 점
movie: Blind Swordsman: Zatoichi, The (Zatôichi) (2003) -> 평균 평점: 3.958 점
movie: Chronicles of Riddick, The (2004) -> 평균 평점: 3.304 점
movie: Garfield: The Movie (2004) -> 평균

movie: Carpool (1996) -> 평균 평점: 1.75 점
movie: Testament of Dr. Mabuse, The (Das Testament des Dr. Mabuse) (1933) -> 평균 평점: 4.0 점
movie: Idiots and Angels (2008) -> 평균 평점: 5.0 점
movie: Flash Gordon Conquers the Universe (1940) -> 평균 평점: 1.25 점
movie: Incendies (2010) -> 평균 평점: 4.4 점
movie: Soul Surfer (2011) -> 평균 평점: 3.5 점
movie: Something Borrowed (2011) -> 평균 평점: 3.25 점
movie: Bridesmaids (2011) -> 평균 평점: 3.762 점
movie: Priest (2011) -> 평균 평점: 3.0 점
movie: Man Escaped, A (Un  condamné à mort s'est échappé ou Le vent souffle où il veut) (1956) -> 평균 평점: 3.5 점
movie: Miracle of Marcelino, The (Marcelino pan y vino) (1955) -> 평균 평점: 3.5 점
movie: Mothra (Mosura) (1961) -> 평균 평점: 3.0 점
movie: Pirates of the Caribbean: On Stranger Tides (2011) -> 평균 평점: 3.229 점
movie: Midnight in Paris (2011) -> 평균 평점: 3.56 점
movie: The Man from Nowhere (2010) -> 평균 평점: 4.5 점
movie: Tree of Life, The (2011) -> 평균 평점: 4.0 점
movie: Kansas City (1996) -> 평균 평점: 3.667 점
movie: Slaughterhouse-Five (1972) -> 평균 

movie: Tarnation (2003) -> 평균 평점: 4.0 점
movie: Pearl Jam Twenty (2011) -> 평균 평점: 0.5 점
movie: I Don't Know How She Does It (2011) -> 평균 평점: 1.5 점
movie: Final Cut, The (2004) -> 평균 평점: 3.667 점
movie: Shark Night 3D (2011) -> 평균 평점: 1.5 점
movie: Being Julia (2004) -> 평균 평점: 1.75 점
movie: Surviving Christmas (2004) -> 평균 평점: 2.75 점
movie: Grudge, The (2004) -> 평균 평점: 2.3 점
movie: Contagion (2011) -> 평균 평점: 3.708 점
movie: Alfie (2004) -> 평균 평점: 2.333 점
movie: Sideways (2004) -> 평균 평점: 3.744 점
movie: Moneyball (2011) -> 평균 평점: 3.885 점
movie: The Machinist (2004) -> 평균 평점: 3.987 점
movie: Vera Drake (2004) -> 평균 평점: 3.0 점
movie: Falling Angels (2003) -> 평균 평점: 3.5 점
movie: Lightning in a Bottle (2004) -> 평균 평점: 4.0 점
movie: Undertow (2004) -> 평균 평점: 3.0 점
movie: Saw (2004) -> 평균 평점: 3.182 점
movie: Ray (2004) -> 평균 평점: 3.684 점
movie: Neds (2010) -> 평균 평점: 4.0 점
movie: Cold Fish (Tsumetai nettaigyo) (2010) -> 평균 평점: 4.5 점
movie: Phineas and Ferb the Movie: Across the 2nd Dimension (2011) -> 평균

movie: To Catch a Thief (1955) -> 평균 평점: 4.217 점
movie: Trailer Park Boys (1999) -> 평균 평점: 5.0 점
movie: This Means War (2012) -> 평균 평점: 2.636 점
movie: John Carter (2012) -> 평균 평점: 3.091 점
movie: Father of the Bride (1950) -> 평균 평점: 3.25 점
movie: Goon (2011) -> 평균 평점: 3.25 점
movie: Band Wagon, The (1953) -> 평균 평점: 3.833 점
movie: Ledge, The (2011) -> 평균 평점: 2.5 점
movie: 21 Jump Street (2012) -> 평균 평점: 3.865 점
movie: Jeff, Who Lives at Home (2012) -> 평균 평점: 4.0 점
movie: Lockout (2012) -> 평균 평점: 3.167 점
movie: Snowtown (Snowtown Murders, The) (2011) -> 평균 평점: 2.5 점
movie: Ninotchka (1939) -> 평균 평점: 4.0 점
movie: Space Battleship Yamato (2010) -> 평균 평점: 3.0 점
movie: Love in the Afternoon (1957) -> 평균 평점: 3.5 점
movie: Jiro Dreams of Sushi (2011) -> 평균 평점: 4.1 점
movie: Damsels in Distress (2011) -> 평균 평점: 2.25 점
movie: Salmon Fishing in the Yemen (2011) -> 평균 평점: 1.5 점
movie: Wrath of the Titans (2012) -> 평균 평점: 2.625 점
movie: Detachment (2011) -> 평균 평점: 4.0 점
movie: Gigi (1958) -> 평균 평점: 3.25

movie: African Queen, The (1951) -> 평균 평점: 3.971 점
movie: House at the End of the Street (2012) -> 평균 평점: 4.0 점
movie: My Left Eye Sees Ghosts (Ngo joh aan gin diy gwai) (2002) -> 평균 평점: 5.0 점
movie: Love Lasts Three Years (L'amour dure trois ans) (2011) -> 평균 평점: 3.5 점
movie: Tall Man, The (2012) -> 평균 평점: 1.5 점
movie: LOL (2012) -> 평균 평점: 3.5 점
movie: Hate (Haine, La) (1995) -> 평균 평점: 3.9 점
movie: Beat the Devil (1953) -> 평균 평점: 3.5 점
movie: Rust and Bone (De rouille et d'os) (2012) -> 평균 평점: 0.5 점
movie: Cat on a Hot Tin Roof (1958) -> 평균 평점: 4.1 점
movie: Marley (2012) -> 평균 평점: 3.5 점
movie: Frankenweenie (2012) -> 평균 평점: 2.5 점
movie: Sinister (2012) -> 평균 평점: 4.0 점
movie: Thing: Terror Takes Shape, The (1998) -> 평균 평점: 1.0 점
movie: Hotel Transylvania (2012) -> 평균 평점: 3.409 점
movie: Side by Side (2012) -> 평균 평점: 3.0 점
movie: Take Aim at the Police Van (Sono gosôsha wo nerae: 'Jûsangô taihisen' yori) (1960) -> 평균 평점: 3.5 점
movie: Meet John Doe (1941) -> 평균 평점: 3.9 점
movie: Argo (2012

##  평균 평점이 가장 높은 영화의 제목

In [639]:
movie_mean = np.array(sorted(list(movie_dict.values()), key=lambda x: x[1]))

In [640]:
high_movie = movie_mean[movie_mean[:,0] == str(movie_mean[:,0].astype(np.float64).max())]

### 정답 

In [643]:
for movie in high_movie:
    print(f'movie: {movie[1]} -> 평균 평점: {round(movie[0].astype(np.float64),3)} 점')

movie: 'Salem's Lot (2004) -> 평균 평점: 5.0 점
movie: 12 Angry Men (1997) -> 평균 평점: 5.0 점
movie: 12 Chairs (1976) -> 평균 평점: 5.0 점
movie: 20 Million Miles to Earth (1957) -> 평균 평점: 5.0 점
movie: 61* (2001) -> 평균 평점: 5.0 점
movie: 7 Faces of Dr. Lao (1964) -> 평균 평점: 5.0 점
movie: 9/11 (2002) -> 평균 평점: 5.0 점
movie: A Detective Story (2003) -> 평균 평점: 5.0 점
movie: A Flintstones Christmas Carol (1994) -> 평균 평점: 5.0 점
movie: A Perfect Day (2015) -> 평균 평점: 5.0 점
movie: A Plasticine Crow (1981) -> 평균 평점: 5.0 점
movie: Act of Killing, The (2012) -> 평균 평점: 5.0 점
movie: Adventures Of Sherlock Holmes And Dr. Watson: The Twentieth Century Approaches (1986) -> 평균 평점: 5.0 점
movie: Advise and Consent (1962) -> 평균 평점: 5.0 점
movie: Alesha Popovich and Tugarin the Dragon (2004) -> 평균 평점: 5.0 점
movie: Alien Contamination (1980) -> 평균 평점: 5.0 점
movie: All Yours (2016) -> 평균 평점: 5.0 점
movie: All the Vermeers in New York (1990) -> 평균 평점: 5.0 점
movie: American Friend, The (Amerikanische Freund, Der) (1977) -> 평균 평점: 5